In [2]:
import xmltodict

In [3]:
class tune:
    
    def __init__(self):
        self.dic = {}
        self.tick = 1
        self.dic['CsoundSynthesizer'] = {}
        self.dic['CsoundSynthesizer']['CsOptions'] =  """
            -o dac
            """
        self.dic['CsoundSynthesizer']['CsInstruments'] =  """
            sr     = 44100     ; Samplerate 44100 Hz (Signalrate für die Ausgabe)
            ksmps  = 32        ; Anzahl von Samples in einem Audio-Array (Blockgröße)
            nchnls = 2         ; Anzahl der Ausgabekanäle (1 bedeutet Mono, 2 Stereo)
            0dbfs = 1
        """
        
        self.dic['CsoundSynthesizer']['CsScore'] =  """
        """
        
        self.scores = {}
        
        # frequency lookup
        factor = 2 ** (1/12)
        notes = 'c c# d d# e f f# g g# a a# b'.split()
        self.freq_dic = {}
        for i, s in enumerate(notes):
            self.freq_dic[s] = 440 * factor ** (i-9)
        
    def add_instr(self, name, s):
        self.dic['CsoundSynthesizer']['CsInstruments'] += f"\ninstr {name}\n{s}\nendin\n"
        
    def add_score(self, score_name, score_text):
        self.scores[score_name] = score_text
        
    def add_track(self, track):
        self.track = track
            
    def set_tempo(self, tempo):
        self.dic['CsoundSynthesizer']['CsScore'] += f't 0 {tempo}\n'
        
    def get_freq(self, note):
        note_name = note[0:-1]
        octave = eval(note[-1]) - 4
        freq = self.freq_dic[note_name]
        return freq * (2 ** octave)
    
    def get_track(self):
        csound = '\n'
        ltrack = self.track.strip().split('\n')
        start = 0
        for strack in ltrack:
            instr, score_name = strack.split(':')
            instr = instr.strip()
            score_name = score_name.strip()
            s, start = self.get_csound(instr, score_name, start)
            csound += s
        return csound
            
    def get_csound(self, instr, score_name, start):
        csound = ''
        score = self.scores[score_name]
        lscore = score.strip().split('\n')  
        for i, line in enumerate(lscore):
            sl = line.split()
 
            offset = eval(sl[0])
            note = sl[1]
            duration = offset
            freq = self.get_freq(note)
            
            volume = 0.8
            csound += f'i "{instr}" {start} {duration} {freq} {volume}\n'
            start = start + offset
        return csound, start
        
    def write(self, file):
        csound = self.get_track()
        self.dic['CsoundSynthesizer']['CsScore'] += csound
        s = xmltodict.unparse(self.dic, pretty=True)

        with open(file, 'w', encoding='utf-8') as fout:
            fout.write(s)

In [28]:
a = tune()
a.add_instr('synth1',"""
    aOut vco2 p5, p4
    out  aOut
""")
a.set_tempo(40)
s = ''
s += """\
    1/8 e5
    1/8 d5
    1/4 f#4
    1/4 g#4
    1/8 c#5
    1/8 b4
    1/4 d4
    1/4 e4
    1/8 b4
    1/8 a4
    1/4 c#4
    1/4 e4
    3/4 a4
"""
a.add_score('motiv1', s)
a.add_track("""
    synth1:motiv1
""")
a.write('test.csd')

In [29]:
!csound test.csd

0dBFS level = 32768.0
--Csound version 6.12 beta (double samples) Oct 24 2018
[commit: 037becccce26dc9ebc37198a75877e817431d978]
libsndfile-1.0.29pre1
UnifiedCSD:  test.csd
STARTING FILE
Creating options
Creating orchestra
closing tag
Creating score
rtaudio: PortAudio module enabled ...
using callback interface
rtmidi: PortMIDI module enabled
instr synth1 uses instrument number 1
Elapsed time at end of orchestra compile: real: 0.004s, CPU: 0.004s
sorting score ...
	... done
Elapsed time at end of score sort: real: 0.004s, CPU: 0.004s
graphics suppressed, ascii substituted
0dBFS level = 1.0
orch now loaded
audio buffered in 4096 sample-frame blocks
PortAudio V19.6.0-devel, revision unknown
   0: dac0 (Microsoft Sound Mapper - Output [MME])
   1: dac1 (Speakers (Realtek(R) Audio) [MME])
   2: dac2 (Primary Sound Driver [Windows DirectSound])
   3: dac3 (Speakers (Realtek(R) Audio) [Windows DirectSound])
   4: dac4 (Speakers (Realtek(R) Audio) [Windows WASAPI])
   5: dac5 (Headphones () [

time resolution is 935.673 ns
